### Importer les librairies de machine learning

In [85]:
import numpy as np
import pandas as pd

import poutyne as pt
from poutyne import Model
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split

### Ajuster les hyperparamètres

In [86]:
test_ratio = 0.2
valid_ratio = 0.2

hidden_size = 512
output_size = 8
dropout = 0.0

n_epochs = 50
batch_size = 32
learning_rate = 0.1
min_lr=0.00001
momentum = 0.9

gamma = 0.1
patience = 1

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Charger le dataframe traité, le séparer en datasets et créer les DataLoaders

In [87]:
# Load data/dataframe.csv file as a pandas dataframe
df = pd.read_csv('data/dataframe.csv')

input_size = len(df.columns) - 2 # Retirer le ID et l'étiquette

# Convert into a tensor
dataset = torch.tensor(df.values)

# Split into train and test
train_data, test_data = train_test_split(dataset, test_size=test_ratio)

# Split train into train and validation
train_data, val_data = train_test_split(train_data, test_size=valid_ratio)

# Split train into X and y without the firs column
X_train, y_train = train_data[:, 1:-1].float(), train_data[:, -1].long()
X_val, y_val = val_data[:, 1:-1].float(), val_data[:, -1].long()
X_test, y_test = test_data[:, 1:-1].float(), test_data[:, -1].long()

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

### Créer l'algorithme FC

In [88]:
class FCNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(FCNetwork, self).__init__()
        self.dropout1 = nn.Dropout(dropout)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)

        hidden_size_2 = hidden_size
        self.dropout2 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_size, hidden_size_2)
        self.bn2 = nn.BatchNorm1d(hidden_size_2)

        hidden_size_3 = hidden_size // 2
        self.dropout3 = nn.Dropout(dropout)
        self.fc3 = nn.Linear(hidden_size_2, hidden_size_3)
        self.bn3 = nn.BatchNorm1d(hidden_size_3)

        hidden_size_4 = hidden_size // 4
        self.dropout4 = nn.Dropout(dropout)
        self.fc4 = nn.Linear(hidden_size_3, hidden_size_4)
        self.bn4 = nn.BatchNorm1d(hidden_size_4)

        hidden_size_5 = hidden_size // 8
        self.dropout5 = nn.Dropout(dropout)
        self.fc5 = nn.Linear(hidden_size_4, hidden_size_5)
        self.bn5 = nn.BatchNorm1d(hidden_size_5)

        self.fc6 = nn.Linear(hidden_size_5, output_size)

    def forward(self, x):
        x = self.dropout1(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout2(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout3(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout4(x)
        x = F.relu(self.bn4(self.fc4(x)))
        x = self.dropout5(x)
        x = F.relu(self.bn5(self.fc5(x)))
        x = self.fc6(x)
        y = F.log_softmax(x, dim=1)
        return y

### Créer le modèle Poutyne

In [89]:
network = FCNetwork(input_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
scheduler = pt.ReduceLROnPlateau(monitor="val_acc", mode='max', patience=patience, factor=gamma, min_lr=min_lr, verbose=True)
model = Model(network, optimizer, criterion, device=device, batch_metrics=["accuracy"])

### Entraîner le modèle sur les données d'entraînement

In [90]:
history = model.fit_generator(train_loader, val_loader, epochs=n_epochs, callbacks=[scheduler])
torch.cuda.empty_cache()

Epoch:  1/50 Train steps: 1500 Val steps: 375 7.55s loss: 1.529484 acc: 44.647248 val_loss: 1.363269 val_acc: 50.012501
Epoch:  2/50 Train steps: 1500 Val steps: 375 6.78s loss: 1.331536 acc: 51.793974 val_loss: 1.189443 val_acc: 56.013001
Epoch:  3/50 Train steps: 1500 Val steps: 375 6.87s loss: 1.211239 acc: 55.763220 val_loss: 1.152754 val_acc: 57.896491
Epoch:  4/50 Train steps: 1500 Val steps: 375 6.83s loss: 1.137223 acc: 58.719840 val_loss: 1.059133 val_acc: 60.580048
Epoch:  5/50 Train steps: 1500 Val steps: 375 6.89s loss: 1.080654 acc: 60.609660 val_loss: 1.038719 val_acc: 61.221768
Epoch:  6/50 Train steps: 1500 Val steps: 375 6.83s loss: 1.037085 acc: 62.041088 val_loss: 0.974548 val_acc: 63.863655
Epoch:  7/50 Train steps: 1500 Val steps: 375 7.24s loss: 0.993169 acc: 63.557945 val_loss: 0.981465 val_acc: 62.988582
Epoch:  8/50 Train steps: 1500 Val steps: 375 7.22s loss: 0.964688 acc: 64.539317 val_loss: 0.939941 val_acc: 64.963747
Epoch:  9/50 Train steps: 1500 Val steps

### Évaluer le modèle sur les données de test

In [91]:
loss, acc = model.evaluate_generator(test_loader)

Test steps: 469 0.86s test_loss: 0.677585 test_acc: 79.191946                                                 
